In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("StructuredStreamingExample") \
    .getOrCreate()

# Step 2: Define Input Source (Socket Streaming)
# Replace 'localhost' and '9999' with your host and port
input_stream = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Step 3: Data Transformation
# Split the lines into words, explode the array of words into rows
words = input_stream.select(
    explode(split(input_stream.value, " ")).alias("word")
)

# Count the occurrences of each word
word_counts = words.groupBy("word").count()

# Step 4: Define Output Sink
# Write the output to the console
query = word_counts.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

# Step 5: Start the Query and Await Termination
query.awaitTermination()


24/11/25 23:10:24 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.33 instead (on interface en0)
24/11/25 23:10:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 23:10:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/25 23:10:25 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/11/25 23:10:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/69/7y9q1qt903vgnyr47cn_dbvm0000gn/T/temporary-03ff6a7a-8698-40c9-84a5-e7173f066fa3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCh

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|Hello|    1|
|world|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|Hellow|    1|
| Hello|    1|
| world|    1|
| Spark|    1|
+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|Hellow|    1|
| Hello|    2|
| spark|    1|
| world|    1|
| Spark|    1|
+------+-----+



24/11/25 23:19:59 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("CatFactsProcessing") \
    .getOrCreate()

# Step 2: Define Input Source (File Stream)
cat_facts_stream = spark.readStream \
    .format("text") \
    .option("path", "output").option("maxFilesPerTrigger", 1).load()

# Step 3: Define Schema for JSON Parsing
cat_fact_schema = StructType() \
    .add("fact", StringType()) \
    .add("length", StringType())

# Step 4: Parse and Transform Data
parsed_stream = cat_facts_stream.select(
    from_json(col("value"), cat_fact_schema).alias("data")
).select("data.fact", "data.length")

# Step 5: Define Output Sink
query = parsed_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Step 6: Start the Query and Await Termination
query.awaitTermination()


24/11/25 23:44:08 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.33 instead (on interface en0)
24/11/25 23:44:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 23:44:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/25 23:44:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/69/7y9q1qt903vgnyr47cn_dbvm0000gn/T/temporary-e8387a1e-3a32-4b8f-89ad-ef0ea94a6fcc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/11/25 23:44:10 WARN ResolveWriteToStream: spark.sq

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+------+
|                fact|length|
+--------------------+------+
|Cat bites are mor...|    60|
+--------------------+------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+------+
|                fact|length|
+--------------------+------+
|During the Middle...|   235|
+--------------------+------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+------+
|                fact|length|
+--------------------+------+
|Purring does not ...|   129|
+--------------------+------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+------+
|                fact|length|
+--------------------+------+
|Cats eat grass to...|    92|
+--------------------+------+

--------

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/bhland/miniforge3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=82>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bhland/miniforge3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/bhland/miniforge3/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/bhland/miniforge3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.

Py4JError: An error occurred while calling o45.awaitTermination